In [4]:
import cv2, os, numpy as np
from math import pi

INPUT_DIR = "fotos"
OUTPUT_DIR = "fotos_binarizadas_limpias"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# parámetros
LEFT_MASK_FRAC = 0.08        # enmascara el 8% izquierdo (cámbialo si hay borde brillante)
BLUR_KSIZE = 7               # suavizado para reducir ruido (impar)
OPEN_K = 5                   # apertura morfológica para quitar puntos sueltos
AREA_MIN = 150               # descarta blobs muy pequeños
CIRC_MIN = 0.5               # circularidad mínima 4πA/P^2 (0..1)

def keep_largest_circular(blob_mask):
    # devuelve una máscara con el blob más grande y más circular
    cnts, _ = cv2.findContours(blob_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    best = None
    best_score = 0.0
    h, w = blob_mask.shape
    for c in cnts:
        area = cv2.contourArea(c)
        if area < AREA_MIN: 
            continue
        peri = cv2.arcLength(c, True) or 1.0
        circ = (4.0 * pi * area) / (peri * peri)  # 1.0 = círculo perfecto
        score = area * (0.5 + 0.5 * min(max(circ, 0.0), 1.0))  # pondera por circularidad y área
        if circ >= CIRC_MIN and score > best_score:
            best = c
            best_score = score
    out = np.zeros((h, w), np.uint8)
    if best is not None:
        cv2.drawContours(out, [best], -1, 255, thickness=-1)
    return out

for subdir, _, files in os.walk(INPUT_DIR):
    rel = os.path.relpath(subdir, INPUT_DIR)
    out_sub = os.path.join(OUTPUT_DIR, rel)
    os.makedirs(out_sub, exist_ok=True)

    for f in files:
        if not f.lower().endswith(".jpg"):
            continue
        in_path  = os.path.join(subdir, f)
        out_path = os.path.join(out_sub, f)

        img = cv2.imread(in_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print("⚠️ No se pudo leer", in_path); continue

        h, w = img.shape

        # (1) Enmascara franja izquierda si hay luz parásita
        if LEFT_MASK_FRAC > 0:
            m = int(w * LEFT_MASK_FRAC)
            bg = int(img[:, m:m+20].mean()) if m+20 < w else int(img.mean())
            img[:, :m] = bg

        # (2) Suavizado para reducir ruido
        blur = cv2.GaussianBlur(img, (BLUR_KSIZE, BLUR_KSIZE), 0)

        # (3) Otsu (la pelota es brillante → queremos binario con pelota blanca)
        _, bin0 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # (4) Morfología: apertura para quitar puntos sueltos
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (OPEN_K, OPEN_K))
        opened = cv2.morphologyEx(bin0, cv2.MORPH_OPEN, k, iterations=1)

        # (5) Quédate con el blob más grande y circular (la pelota)
        clean = keep_largest_circular(opened)

        cv2.imwrite(out_path, clean)
        print(f"✅ {in_path} -> {out_path}")

print("✨ Proceso completado")

✅ fotos/d650b919-572d-4239-abe6-87898764321a/14.jpg -> fotos_binarizadas_limpias/d650b919-572d-4239-abe6-87898764321a/14.jpg
✅ fotos/d650b919-572d-4239-abe6-87898764321a/15.jpg -> fotos_binarizadas_limpias/d650b919-572d-4239-abe6-87898764321a/15.jpg
✅ fotos/d650b919-572d-4239-abe6-87898764321a/17.jpg -> fotos_binarizadas_limpias/d650b919-572d-4239-abe6-87898764321a/17.jpg
✅ fotos/d650b919-572d-4239-abe6-87898764321a/16.jpg -> fotos_binarizadas_limpias/d650b919-572d-4239-abe6-87898764321a/16.jpg
✅ fotos/d650b919-572d-4239-abe6-87898764321a/20.jpg -> fotos_binarizadas_limpias/d650b919-572d-4239-abe6-87898764321a/20.jpg
✅ fotos/d650b919-572d-4239-abe6-87898764321a/18.jpg -> fotos_binarizadas_limpias/d650b919-572d-4239-abe6-87898764321a/18.jpg
✅ fotos/d650b919-572d-4239-abe6-87898764321a/19.jpg -> fotos_binarizadas_limpias/d650b919-572d-4239-abe6-87898764321a/19.jpg
✅ fotos/314a7bb1-8535-48e4-9abd-bf6927a87952/14.jpg -> fotos_binarizadas_limpias/314a7bb1-8535-48e4-9abd-bf6927a87952/14.jpg


In [3]:
import os

INPUT_DIR = "fotos_binarizadas_limpias"

# Lista de subcarpetas directas
subfolders = [d for d in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR, d))]

print(f"📂 Número de subcarpetas: {len(subfolders)}")
print("➡️ Subcarpetas encontradas:", subfolders)

📂 Número de subcarpetas: 264
➡️ Subcarpetas encontradas: ['efdb9a78-d21c-4d28-9ffa-d18b0da9c036', 'd69bd30a-af3d-4f59-a594-9b048191a93c', 'c816574e-c150-471a-819c-ac50a86ed1e4', '716251da-fee2-4967-b9fe-7866a31ef411', 'ad89c9ed-55d0-4d58-ae1b-342d2b148419', '37b80824-be09-40e3-a740-324b76a4c4df', '9478fe9b-e2ca-4352-b505-bda81dfa7999', 'f90398bd-2c43-48d0-9793-a6afa39af3f5', 'bb497379-c3fe-4c57-828e-50373b83e0ac', 'e108701f-ac43-4def-95d9-2bbd7e64ce0a', '6db94e04-74cb-40bb-8540-e6d85c131d4b', 'd48872f4-b92f-47f7-bc51-e50d59c2c20a', '9235ab67-d77d-4765-87ec-9767ac18128b', '0ea9061b-6924-4f0a-a41c-83c3641d5a1a', '037d3d2a-f9e1-42d3-8264-4a55b78919f5', '39e5d3ac-74c8-4519-87fb-597a63dc808e', '8fa10762-8053-4cbe-ba9d-891251fae125', '64c45ea5-7d6a-4d65-9beb-277d4096c980', 'af8e0873-efd6-49f5-aab8-5e2c40391230', 'b831fa19-6734-4c33-8469-368caa5efb0d', 'b3c46160-4170-4068-9216-840f5096412b', 'f2ed4b19-b651-4371-8184-b34c49fc1640', 'ae167887-866f-432f-9ba4-00f3eecd4d1c', '3241265e-3cbf-451d-a7